# <font color="#418FDE" size="6.5" uppercase>**DP Fundamentals**</font>

>Last update: 20260102.
    
By the end of this Lecture, you will be able to:
- Identify overlapping subproblems and optimal substructure in candidate problems. 
- Derive recurrence relations that define dynamic programming solutions. 
- Implement memoized recursive solutions in Python to improve performance over naive recursion. 


## **1. Core DP Concepts**

### **1.1. Recognizing Repeated Subproblems**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master Python Algorithms/Module_08/Lecture_A/image_01_01.jpg?v=1767343875" width="250">



>* Break problems into smaller tasks and compare them
>* Reuse answers to identical subproblems to save work

>* Sketch naive recursion and watch repeated states
>* Same state means same subproblem; avoid recomputation

>* Real-world decisions often reuse the same state
>* Recognizing repeated states enables efficient dynamic programming reuse



In [ ]:
#@title Python Code - Recognizing Repeated Subproblems

# Demonstrates repeated subproblems using naive Fibonacci recursion.
# Shows how the same inputs are recomputed many different times.
# Helps you visually recognize overlapping subproblems in recursion trees.

# pip install commands are unnecessary because this script uses only standard libraries.

# Import functools for a simple call counter decorator.
import functools

# Define a decorator that counts function calls by input value.
def count_calls(func):

    # Store counts inside a dictionary keyed by argument value.
    counts = {}

    # Wrap the original function with counting behavior.
    @functools.wraps(func)
    def wrapper(n):

        # Increase the count for this specific argument value.
        counts[n] = counts.get(n, 0) + 1

        # Call the original function and return its result.
        return func(n)

    # Attach counts dictionary so we can inspect it later.
    wrapper.counts = counts

    # Return the wrapped function with counting capability.
    return wrapper

# Define naive recursive Fibonacci to create overlapping subproblems.
@count_calls
def fib(n):

    # Base case for n equal zero or one returns n directly.
    if n <= 1:
        return n

    # Recursive case splits into two smaller Fibonacci subproblems.
    return fib(n - 1) + fib(n - 2)

# Choose a small n to keep recursion manageable and understandable.
n = 6

# Compute Fibonacci of n using the naive recursive function.
result = fib(n)

# Print the final Fibonacci result for the chosen n value.
print("fib(", n, ") =", result)

# Print how many times each subproblem fib(k) was recomputed.
print("Call counts for each subproblem input:")

# Loop through sorted keys to show repeated subproblem evaluations clearly.
for k in sorted(fib.counts.keys()):

    # Print each input value and its corresponding call count.
    print("fib(", k, ") was called", fib.counts[k], "times")



### **1.2. Building Optimal Solutions**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master Python Algorithms/Module_08/Lecture_A/image_01_02.jpg?v=1767343891" width="250">



>* Optimal solutions come from combining optimal subproblems
>* Like cheapest trip built from cheapest segments

>* Define problem states that capture necessary information
>* Relate each state’s best choice to smaller states

>* Local best choices can harm global results
>* Independent subproblems let optimal pieces combine safely



In [ ]:
#@title Python Code - Building Optimal Solutions

# Demonstrate building optimal solutions from smaller optimal parts using dynamic programming.
# Show how each route segment cost is reused when building the full trip cost.
# Compare naive recursive computation with memoized dynamic programming computation for clarity.
# !pip install some_required_library_here.

# Define road trip segments with distances in miles and fuel cost per mile.
segments = {('A', 'B'): 100, ('B', 'C'): 150, ('A', 'C'): 260}

# Define a simple function computing trip cost using a fixed fuel price.
fuel_price_per_mile = 0.25

# Define a naive recursive function computing minimal cost between two cities.
def min_cost_naive(start, end):
    if (start, end) in segments:
        return segments[(start, end)] * fuel_price_per_mile
    if start == 'A' and end == 'C':
        cost_via_B = min_cost_naive('A', 'B') + min_cost_naive('B', 'C')
        return cost_via_B
    return float('inf')

# Define a memoized function storing optimal subproblem solutions for reuse.
memo = {}

# Define a dynamic programming function using memoization for repeated subproblems.
def min_cost_dp(start, end):
    key = (start, end)
    if key in memo:
        return memo[key]
    if key in segments:
        memo[key] = segments[key] * fuel_price_per_mile
        return memo[key]
    if start == 'A' and end == 'C':
        cost_via_B = min_cost_dp('A', 'B') + min_cost_dp('B', 'C')
        memo[key] = cost_via_B
        return memo[key]
    memo[key] = float('inf')
    return memo[key]

# Compute and print naive minimal cost for the full trip from A to C.
naive_total = min_cost_naive('A', 'C')

# Compute and print dynamic programming minimal cost for the full trip.
dp_total = min_cost_dp('A', 'C')

# Print results showing both methods build from optimal segment solutions.
print('Naive minimal cost from A to C equals', round(naive_total, 2))

# Print dynamic programming result and confirm equality with naive computation.
print('DP minimal cost from A to C equals', round(dp_total, 2))



### **1.3. Recursion to Dynamic Programming**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master Python Algorithms/Module_08/Lecture_A/image_01_03.jpg?v=1767343907" width="250">



>* Start with recursion to break problems into subproblems
>* Notice repeated subproblems and reuse results with DP

>* View recursion as overlapping subproblems solved once
>* Index subproblems in a table to reuse results

>* Use recursion to spot repeating intermediate questions
>* Store and reuse states to scale solutions



In [ ]:
#@title Python Code - Recursion to Dynamic Programming

# Demonstrate recursion transforming into dynamic programming with memoization for stair climbing.
# Compare naive recursive calls against memoized dynamic programming calls for same problem.
# Show reduced repeated work by counting function calls for both solution strategies.

# !pip install example_library_not_needed_but_placeholder_only.

# Import functools for memoization decorator usage in dynamic programming version.
import functools

# Define naive recursive function counting ways to climb stairs with one or two steps.
def ways_naive(steps_remaining, counter):
    # Increment naive call counter each time function executes for any argument.
    counter[0] += 1
    # Handle base case when no steps remain meaning one valid climbing way.
    if steps_remaining == 0:
        return 1
    # Handle base case when steps become negative meaning no valid climbing way.
    if steps_remaining < 0:
        return 0
    # Recursively compute ways by taking one step then two steps from current position.
    return ways_naive(steps_remaining - 1, counter) + ways_naive(steps_remaining - 2, counter)

# Define memoized dynamic programming version using least recently used cache decorator.
@functools.lru_cache(maxsize=None)
def ways_dp(steps_remaining):
    # Handle base case when no steps remain meaning one valid climbing way.
    if steps_remaining == 0:
        return 1
    # Handle base case when steps become negative meaning no valid climbing way.
    if steps_remaining < 0:
        return 0
    # Recursively compute ways using cached overlapping subproblem results when available.
    return ways_dp(steps_remaining - 1) + ways_dp(steps_remaining - 2)

# Choose total stairs count representing a small staircase measured in feet length.
stairs_count = 10

# Initialize list counter for naive calls since integers are immutable in Python.
naive_counter = [0]

# Compute naive result and track how many recursive calls were executed overall.
naive_result = ways_naive(stairs_count, naive_counter)

# Compute dynamic programming result using memoization for same staircase problem.
dp_result = ways_dp(stairs_count)

# Count how many distinct subproblems were solved using cached dynamic programming.
dp_calls = ways_dp.cache_info().hits + ways_dp.cache_info().misses

# Print comparison showing identical answers but drastically fewer dynamic programming calls.
print("Stairs:", stairs_count, "Naive ways:", naive_result, "DP ways:", dp_result)

# Print naive recursion call count representing full recursion tree size without memoization.
print("Naive recursion calls total:", naive_counter[0])

# Print dynamic programming call count representing distinct overlapping subproblems solved once.
print("Dynamic programming calls total:", dp_calls)



## **2. Designing Recurrences**

### **2.1. Subproblem State Design**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master Python Algorithms/Module_08/Lecture_A/image_02_01.jpg?v=1767343925" width="250">



>* Define each subproblem with a precise snapshot
>* Include only information needed for optimal next decisions

>* Choose minimal information needed to finish optimally
>* State makes future decisions independent of past path

>* Balance enough information with manageable state size
>* Iteratively refine states to capture only essentials



### **2.2. Formulating Recurrence Relations**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master Python Algorithms/Module_08/Lecture_A/image_02_02.jpg?v=1767343935" width="250">



>* Define how each state uses smaller solutions
>* Connect current decisions to previously solved subproblems

>* Center recurrence on key choices or boundaries
>* Map each choice outcome to a defined subproblem

>* Separate allowed, forbidden, cheap, and expensive transitions
>* Encode constraints so DP handles complex real decisions



### **2.3. Validating Base Cases**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master Python Algorithms/Module_08/Lecture_A/image_02_03.jpg?v=1767343952" width="250">



>* Base cases are essential, smallest clearly defined problems
>* Derive base cases from plain-language state meaning

>* Test base cases with tiny, concrete examples
>* Check edge conditions behave with intuitive, common sense

>* Ensure every recursion path reaches a base case
>* Check indices, empty cases, and logical consistency



In [ ]:
#@title Python Code - Validating Base Cases

# Demonstrate validating base cases in a tiny dynamic programming example.
# Compare correct and incorrect base cases for a simple climbing stairs problem.
# Show how wrong base cases silently change answers for small input values.

# !pip install nothing_needed_here_this_runs_with_standard_python_only.

# Define a recursive function with memoization and correct base cases.
from functools import lru_cache

# This function counts ways to climb n steps using one or two steps.
@lru_cache(maxsize=None)
def ways_correct(n):
    # Base case when n equals zero steps remaining to climb.
    if n == 0:
        return 1
    # Base case when n becomes negative meaning overshooting steps.
    if n < 0:
        return 0
    # Recursive relation using one step or two steps choices.
    return ways_correct(n - 1) + ways_correct(n - 2)

# Define a similar function but with intentionally wrong base cases.
@lru_cache(maxsize=None)
def ways_wrong(n):
    # Wrong base case returning zero for exactly zero steps.
    if n == 0:
        return 0
    # Wrong base case allowing negative steps to still count paths.
    if n < 0:
        return 1
    # Same recurrence relation as the correct function above.
    return ways_wrong(n - 1) + ways_wrong(n - 2)

# Helper function to print comparison for several small step counts.
def compare_small_values(max_n):
    # Print header describing columns for clarity and understanding.
    print("n, correct_ways, wrong_ways")
    # Loop through small n values to test base case behavior.
    for n in range(max_n + 1):
        print(n, ways_correct(n), ways_wrong(n))

# Run comparison for small staircase sizes to validate base cases.
compare_small_values(5)



## **3. Python Memoization Basics**

### **3.1. Dictionary Based Memoization**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master Python Algorithms/Module_08/Lecture_A/image_03_01.jpg?v=1767343974" width="250">



>* Store recursive results in a dictionary notebook
>* Reuse stored answers to avoid repeated subproblems

>* Check dictionary for existing result before recursing
>* Store new results so later calls run faster

>* Memoization speeds up many real-world recursive tasks
>* Keeps recursion elegant while matching DP performance



In [ ]:
#@title Python Code - Dictionary Based Memoization

# Demonstrate dictionary based memoization with a simple Fibonacci example.
# Compare naive recursion and memoized recursion performance using small inputs.
# Show how a dictionary stores and reuses previously computed subproblem results.

# pip install some_required_library_if_needed_but_standard_library_is_sufficient.

# Import time module for simple timing measurements.
import time

# Define naive recursive Fibonacci without memoization.
def fib_naive(n):
    if n <= 1:
        return n
    return fib_naive(n - 1) + fib_naive(n - 2)

# Define memoized Fibonacci using a dictionary cache.
cache = {}

def fib_memo(n):
    if n in cache:
        return cache[n]
    if n <= 1:
        cache[n] = n
        return n
    result = fib_memo(n - 1) + fib_memo(n - 2)
    cache[n] = result
    return result

# Choose a Fibonacci index that shows noticeable speed difference.
n = 30

# Time the naive recursive Fibonacci call.
start_naive = time.time()
value_naive = fib_naive(n)
end_naive = time.time()

# Time the memoized recursive Fibonacci call.
start_memo = time.time()
value_memo = fib_memo(n)
end_memo = time.time()

# Print results and timings for comparison.
print("Naive Fibonacci value and seconds:", value_naive, round(end_naive - start_naive, 4))
print("Memoized Fibonacci value and seconds:", value_memo, round(end_memo - start_memo, 4))
print("Cache dictionary keys count now:", len(cache))



### **3.2. LRU Cache Decorator**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master Python Algorithms/Module_08/Lecture_A/image_03_02.jpg?v=1767343991" width="250">



>* LRU cache decorator automatically remembers function results
>* Speeds up recursion while keeping code simple

>* LRU cache keeps only most recently used results
>* Cache size tuning balances memory use and speed

>* Decorator works best with pure, side‑effect‑free functions
>* Fits DP subproblems and speeds up recursive solutions



In [ ]:
#@title Python Code - LRU Cache Decorator

# Demonstrate using functools lru_cache decorator for simple recursive memoization example.
# Compare naive recursive Fibonacci with cached Fibonacci using lru_cache decorator.
# Show speed difference and identical results using small Fibonacci index values.

# pip install commands are unnecessary because functools exists in standard library.

# Import time module for simple execution time measurements.
import time

# Import lru_cache decorator from functools standard library module.
from functools import lru_cache

# Define naive Fibonacci function without any caching behavior.
def fib_naive(n):
    # Base case returns n when n is zero or one.
    if n <= 1:
        return n
    # Recursive case sums previous two Fibonacci numbers.
    return fib_naive(n - 1) + fib_naive(n - 2)

# Apply lru_cache decorator with limited maximum cache size.
@lru_cache(maxsize=128)
# Define cached Fibonacci function using same mathematical definition.
def fib_cached(n):
    # Base case returns n when n is zero or one.
    if n <= 1:
        return n
    # Recursive case sums previous two Fibonacci numbers.
    return fib_cached(n - 1) + fib_cached(n - 2)

# Choose Fibonacci index that is large enough to show timing difference.
N = 32

# Measure time for naive Fibonacci computation using time.perf_counter.
start_naive = time.perf_counter()

# Compute naive Fibonacci result for chosen index N.
result_naive = fib_naive(N)

# Compute elapsed time for naive Fibonacci computation.
elapsed_naive = time.perf_counter() - start_naive

# Measure time for cached Fibonacci computation using time.perf_counter.
start_cached = time.perf_counter()

# Compute cached Fibonacci result for chosen index N.
result_cached = fib_cached(N)

# Compute elapsed time for cached Fibonacci computation.
elapsed_cached = time.perf_counter() - start_cached

# Print Fibonacci index and both computed results for comparison.
print("Fibonacci index:", N, "naive:", result_naive, "cached:", result_cached)

# Print naive execution time in seconds with simple formatting.
print("Naive time seconds:", round(elapsed_naive, 6))

# Print cached execution time in seconds with simple formatting.
print("Cached time seconds:", round(elapsed_cached, 6))

# Print speedup factor showing how many times faster cached version runs.
print("Speedup factor:", round(elapsed_naive / elapsed_cached, 2))



### **3.3. Measuring speedups empirically**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master Python Algorithms/Module_08/Lecture_A/image_03_03.jpg?v=1767344005" width="250">



>* Run timing tests on naive and memoized versions
>* Measure runtimes across inputs to quantify speedup

>* Run both versions repeatedly and average timings
>* Keep work identical so differences show memoization impact

>* Small inputs hide memoization benefits; large reveal
>* Timing plots show real-world value of memoization



In [ ]:
#@title Python Code - Measuring speedups empirically

# Demonstrate measuring speedups from memoization using simple timing experiments.
# Compare naive recursive Fibonacci with memoized Fibonacci for increasing input sizes.
# Print timing results to see empirical speedups clearly and quantitatively.

# pip install commands are unnecessary because we only use Python standard library.

# Import time module for measuring elapsed execution time precisely.
import time

# Define naive recursive Fibonacci without any memoization cache mechanism.
def fib_naive(n):
    if n <= 1:
        return n
    return fib_naive(n - 1) + fib_naive(n - 2)

# Define memoized Fibonacci using dictionary cache for storing previous results.
memo_cache = {}

def fib_memo(n):
    if n in memo_cache:
        return memo_cache[n]
    if n <= 1:
        memo_cache[n] = n
    else:
        memo_cache[n] = fib_memo(n - 1) + fib_memo(n - 2)
    return memo_cache[n]

# Define helper function for timing another function with given argument value.
def time_call(func, n):
    start = time.perf_counter()
    result = func(n)
    end = time.perf_counter()
    return result, end - start

# Choose input sizes large enough to show clear timing differences.
input_sizes = [20, 25, 30, 32]

# Print table header describing columns for clarity and readability.
print("n  naive_time_s  memo_time_s  same_result")

# Loop over input sizes and measure both implementations systematically.
for n in input_sizes:
    naive_result, naive_time = time_call(fib_naive, n)
    memo_result, memo_time = time_call(fib_memo, n)
    same = naive_result == memo_result
    print(f"{n:<2} {naive_time:>11.6f} {memo_time:>11.6f} {same}")




# <font color="#418FDE" size="6.5" uppercase>**DP Fundamentals**</font>


In this lecture, you learned to:
- Identify overlapping subproblems and optimal substructure in candidate problems. 
- Derive recurrence relations that define dynamic programming solutions. 
- Implement memoized recursive solutions in Python to improve performance over naive recursion. 

In the next Lecture (Lecture B), we will go over 'Tabulation Patterns'